# 1. start with small idea of RAG - search by keywords matching

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import functools
from concurrent.futures import ThreadPoolExecutor
import time

c:\Users\admin\miniconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from dotenv import load_dotenv
import os
load_dotenv()
print("Key loaded:", os.getenv("OPENAI_API_KEY")[:10] + "...")

Key loaded: sk-proj-Ag...


In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

# Configuration

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
import os, glob, functools
from concurrent.futures import ThreadPoolExecutor

@functools.lru_cache(maxsize=None)
def load_context():
    context = {}

    def load_regulation_files():
        regulation_context = {}
        regulations = glob.glob(r"E:\test_data_chatbot\dataset\quy_che_dao_tao\*.md")
        print(f"Found {len(regulations)} files in quy_che_dao_tao")

        def load_single_regulation(regulation):
            # Lấy tên file mà không bị mất dấu cách
            name = os.path.splitext(os.path.basename(regulation))[0]
            try:
                with open(regulation, "r", encoding="utf-8") as f:
                    return name, f.read()
            except Exception as e:
                print(f"Error loading {regulation}: {e}")
                return name, ""

        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(load_single_regulation, regulations)
            regulation_context.update(dict(results))
        return regulation_context

    with ThreadPoolExecutor(max_workers=2) as executor:
        regulation_future = executor.submit(load_regulation_files)
        context.update(regulation_future.result())

    return context


In [5]:
ctx = load_context()
print(ctx.keys())       # Sẽ ra: dict_keys(['Chapter I', 'Chapter II', ...])
print(ctx['Chapter I'][:200])  # In 200 ký tự đầu tiên để xem nội dung load đúng

Found 5 files in quy_che_dao_tao
dict_keys(['Chapter I', 'Chapter II', 'chapter III', 'Chapter IV', 'Chapter V'])
# Chương I: NHỮNG QUY ĐỊNH CHUNG

## Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng
1. Quy chế này quy định đào tạo đại học và cao đẳng hệ chính quy theo học chế  tín chỉ của trường Đại học Công nghi


In [6]:
system_message = (
    "Bạn là một chuyên gia tư vấn đào tạo tại Trường Đại học Công nghiệp Hà Nội. "
    "Nhiệm vụ của bạn là trả lời các câu hỏi liên quan đến quy chế đào tạo đại học và cao đẳng hệ chính quy theo học chế tín chỉ, "
    "bao gồm thời gian đào tạo, kế hoạch học tập, đăng ký học phần, lớp học, và các quy định liên quan khác. "
    "Nếu bạn không biết câu trả lời, hãy nói rõ rằng bạn không biết. "
    "Tuyệt đối không bịa ra thông tin nếu không có ngữ cảnh liên quan được cung cấp."
)


In [7]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in ctx.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context  

In [8]:
import os, glob, re

def load_all_context(folder_path):
    context = {}
    files = glob.glob(os.path.join(folder_path, "*.md"))

    for file in files:
        with open(file, "r", encoding="utf-8") as f:
            content = f.read()

        # Tách "Điều X"
        matches = re.split(r"(Điều\s+\d+)", content)
        for i in range(1, len(matches), 2):
            title = matches[i].strip()   # ví dụ: "Điều 7"
            body = matches[i+1].strip() if i+1 < len(matches) else ""
            context[title] = body

    return context

# Load toàn bộ 5 chương
ctx = load_all_context(r"E:\test_data_chatbot\dataset\quy_che_dao_tao")

print(ctx.keys())  
# -> dict_keys(['Điều 1', 'Điều 2', ..., 'Điều n'])


dict_keys(['Điều 1', 'Điều 2', 'Điều 3', 'Điều 4', 'Điều 5', 'Điều 6', 'Điều 7', 'Điều 8', 'Điều 9', 'Điều 10', 'Điều 11', 'Điều 12', 'Điều 13', 'Điều 14', 'Điều 15', 'Điều 16', 'Điều 31', 'Điều 17', 'Điều 18', 'Điều 19', 'Điều 20', 'Điều 21', 'Điều 22', 'Điều 23', 'Điều 24', 'Điều 25', 'Điều 26', 'Điều 27', 'Điều 28', 'Điều 29', 'Điều 30', 'Điều 32'])


In [9]:
get_relevant_context("Who is Cuong?")

[]

In [10]:
get_relevant_context("what is Điều 1")

['. Phạm vi điều chỉnh và đối tượng áp dụng\n1. Quy chế này quy định đào tạo đại học và cao đẳng hệ chính quy theo học chế  tín chỉ của trường Đại học Công nghiệp Hà Nội, bao gồm các nội dung về: tổ chức đào tạo; kiểm tra và thi học phần; xét và công nhận tốt nghiệp.\n2. Quy chế này áp dụng đối với sinh viên hệ chính quy các trình độ đại học và cao đẳng của trường Đại học Công nghiệp Hà Nội từ học kỳ 2 năm học 2012-2013.\n\n##']

In [11]:
get_relevant_context("what is Chương II and introduce about Điều 10")

['. Phạm vi điều chỉnh và đối tượng áp dụng\n1. Quy chế này quy định đào tạo đại học và cao đẳng hệ chính quy theo học chế  tín chỉ của trường Đại học Công nghiệp Hà Nội, bao gồm các nội dung về: tổ chức đào tạo; kiểm tra và thi học phần; xét và công nhận tốt nghiệp.\n2. Quy chế này áp dụng đối với sinh viên hệ chính quy các trình độ đại học và cao đẳng của trường Đại học Công nghiệp Hà Nội từ học kỳ 2 năm học 2012-2013.\n\n##',
 'của Quy chế này.\n\n- Sinh viên được phép thôi học đối với học phần xin rút bớt sau khi kết quả xin rút được nhà trường chấp thuận.\n\n##']

In [12]:
get_relevant_context("what is Điều 25")

['của Quy chế này;\n\nc) Điểm trung bình chung tích lũy của toàn khóa học đạt từ 2,00 trở lên;\n\nd) Có các chứng chỉ ngoại ngữ, tin học theo quy định của Hiệu trưởng;\n\nđ) Có chứng chỉ giáo dục quốc phòng – an ninh đối với các ngành đào tạo không chuyên về quân sự và hoàn thành học phần giáo dục thể chất đối với các ngành không chuyên về thể dục - thể thao;\n\ne) Có đơn gửi Phòng đào tạo đề nghị được xét tốt nghiệp trong trường hợp đủ điều kiện tốt nghiệp sớm hoặc muộn so với thời gian thiết kế của khoá học.\n\n2. Sau mỗi học kỳ, Hội đồng xét tốt nghiệp căn cứ các điều kiện công nhận tốt nghiệp quy định tại khoản 1 Điều này để lập danh sách những sinh viên đủ điều kiện tốt nghiệp.\n\nHội đồng xét tốt nghiệp trường do Hiệu trưởng hoặc Phó Hiệu trưởng được Hiệu trưởng uỷ quyền làm Chủ tịch, trưởng phòng đào tạo làm Thư ký và các thành viên là các trưởng khoa chuyên môn, trưởng phòng công tác sinh viên.\n\n3. Căn cứ đề nghị của Hội đồng xét tốt nghiệp, Hiệu trưởng ký quyết định công nhậ

In [13]:
def add_context(message):
    """Add relevant context to message"""
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nNhững thông tin sau có thể hữu ích cho việc trả lời câu hỏi này:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [14]:
def chat(message, history):
    """Optimized chat function with better error handling"""
    try:
        messages = [{"role": "system", "content": system_message}] + history
        message = add_context(message)
        messages.append({"role": "user", "content": message})

        stream = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            stream=True,
            max_tokens=1000,  # Limit response length for faster generation
            temperature=0.7
        )

        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content
                yield response
    except Exception as e:
        yield f"Xin lỗi, đã có lỗi xảy ra: {str(e)}"

In [15]:
# Launch first version
# print("Launching keyword-based RAG chatbot...")
# view = gr.ChatInterface(chat, type="messages").launch()

# 2. RAG bigger idea with vector search - optimized version

In [16]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

#folders = glob.glob("test_data_chatbot\*")

#text_loader_kwargs = {'encoding': 'utf-8'}
# Nếu dòng trên không hoạt động, người dùng Windows có thể dùng dòng dưới thay thế
# text_loader_kwargs={'autodetect_encoding': True}

# documents = []
# for folder in folders:
#     doc_type = os.path.basename(folder)
#     loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
#     folder_docs = loader.load()
#     for doc in folder_docs:
#         doc.metadata["doc_type"] = doc_type
#         documents.append(doc)

# print("Total documents loaded:", len(documents))


In [18]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Tùy chọn encoding
text_loader_kwargs = {'autodetect_encoding': True}

# Loader cho toàn bộ thư mục
loader = DirectoryLoader(
    "E:/test_data_chatbot/dataset",       # thư mục gốc
    glob="**/*.md",            # lấy tất cả file .md trong subfolder
    loader_cls=TextLoader,
    loader_kwargs=text_loader_kwargs,
    recursive=True
)

# Load documents
documents = loader.load()

# Thêm metadata = tên file (không extension)
for doc in documents:
    filename = os.path.basename(doc.metadata["source"])
    doc.metadata["doc_type"] = os.path.splitext(filename)[0]

print("Total documents loaded:", len(documents))
for d in documents:
    print(d.metadata)


Total documents loaded: 6
{'source': 'E:\\test_data_chatbot\\dataset\\Quy_che_dao_tao_HaUI.md', 'doc_type': 'Quy_che_dao_tao_HaUI'}
{'source': 'E:\\test_data_chatbot\\dataset\\quy_che_dao_tao\\Chapter I.md', 'doc_type': 'Chapter I'}
{'source': 'E:\\test_data_chatbot\\dataset\\quy_che_dao_tao\\Chapter II.md', 'doc_type': 'Chapter II'}
{'source': 'E:\\test_data_chatbot\\dataset\\quy_che_dao_tao\\chapter III.md', 'doc_type': 'chapter III'}
{'source': 'E:\\test_data_chatbot\\dataset\\quy_che_dao_tao\\Chapter IV.md', 'doc_type': 'Chapter IV'}
{'source': 'E:\\test_data_chatbot\\dataset\\quy_che_dao_tao\\Chapter V.md', 'doc_type': 'Chapter V'}


In [19]:
documents[0]

Document(metadata={'source': 'E:\\test_data_chatbot\\dataset\\Quy_che_dao_tao_HaUI.md', 'doc_type': 'Quy_che_dao_tao_HaUI'}, page_content='| BỘ  CÔNG  THƯƠNG    TRƯỜNG ĐẠI HỌC CÔNG NGHIỆP HÀ NỘI    |      CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM    Độc lập - Tự do - Hạnh phúc   |\n|----------------------------------------------------------|--------------------------------------------------------------------------|\n\n**QUY CHẾ**\n\n**Đào tạo đại học và cao đẳng hệ chính quy theo học chế tín chỉ**\n\n**tại trường Đại học công nghiệp Hà Nội**\n\n*(Ban hành kèm theo Quyết định số* ***530*** */QĐ-ĐHCN ngày  15 tháng 02 năm 2013 của Hiệu trưởng trường Đại học Công nghiệp Hà Nội)*\n\n**Chương I**\n\n**NHỮNG QUY ĐỊNH CHUNG**\n\n**Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng**\n\n1. Quy chế này quy định đào tạo đại học và cao đẳng hệ chính quy theo học chế  tín chỉ của trường Đại học Công nghiệp Hà Nội, bao gồm các nội dung về: tổ chức đào tạo; kiểm tra và thi học phần; xét và công nhận tốt nghiệ

In [20]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Slightly smaller chunks for better retrieval
    chunk_overlap=100,  # Reduced overlap for performance
    separators=["\n\n", "\n", ". ", " ", ""]  # Better separation
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

Created 121 chunks


In [21]:
chunks[9]

Document(metadata={'source': 'E:\\test_data_chatbot\\dataset\\Quy_che_dao_tao_HaUI.md', 'doc_type': 'Quy_che_dao_tao_HaUI'}, page_content='2. Căn cứ vào khối lượng và nội dung kiến thức tối thiểu quy định cho các chương trình, Trưởng phòng đào tạo dự kiến phân bổ số học phần cho từng năm học, từng học kỳ.\n\n3. Thời gian tối đa hoàn thành chương trình bằng 2 lần thời gian thiết kế cho chương trình quy định tại khoản 1 của Điều này.\n\nCác đối tượng được hưởng chính sách ưu tiên theo quy định tại Quy chế tuyển sinh đại học, cao đẳng hệ chính quy không bị hạn chế về thời gian tối đa để hoàn thành chương trình.\n\n**Điều 7. Đăng ký nhập học**')

In [22]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Các loại tài liệu đã tìm thấy: {', '.join(doc_types)}")

Các loại tài liệu đã tìm thấy: Quy_che_dao_tao_HaUI, chapter III, Chapter V, Chapter II, Chapter IV, Chapter I


In [23]:
for chunk in chunks:
    if 'Nguyễn Thị Lan' in chunk.page_content:
        print(chunk)
        print("_________")

In [24]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [25]:
# Đưa các đoạn văn bản (chunks) vào Vector Store, liên kết mỗi đoạn với một vector embedding

#embeddings = OpenAIEmbeddings()

# Nếu bạn muốn sử dụng embeddings miễn phí từ HuggingFace (thay vì OpenAI),
# hãy thay dòng embeddings = OpenAIEmbeddings()
# bằng:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\admin\AppData\Local\Temp\ipykernel_13052\3339664498.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [26]:
# Vì giá cả là yếu tố quan trọng với công ty của chúng ta, nên ta sẽ sử dụng mô hình chi phí thấp
MODEL = "gpt-4o-mini"

# Đặt tên cho database vector (có thể tùy chọn)
db_name = "vector_db"

# Kiểm tra nếu database Chroma đã tồn tại, thì xóa collection để khởi động lại từ đầu
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [27]:
# Tạo vector store bằng Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,              # Danh sách các đoạn văn bản đã chia nhỏ
    embedding=embeddings,          # Hàm embedding (ví dụ: OpenAI hoặc HuggingFace)
    persist_directory=db_name      # Thư mục lưu trữ database
)
# Kiểm tra số lượng document đã được lưu vào vector store
print(f"Vectorstore created with {vectorstore._collection.count()} documents")


Vectorstore created with 121 documents


In [28]:
# Lấy ra bộ sưu tập vector từ vectorstore
collection = vectorstore._collection

# Lấy 1 embedding từ database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# Kiểm tra số chiều (số phần tử trong vector)
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


In [29]:
sample_embedding

array([-7.52693787e-02,  8.09942260e-02,  5.66497911e-03, -5.37944995e-02,
       -3.27837318e-02,  2.05683913e-02,  5.53156212e-02,  2.88856588e-02,
        3.55300121e-02,  2.96965684e-03,  1.97156906e-01, -1.52353927e-01,
       -1.57556459e-02, -3.59294340e-02,  1.93178281e-02,  2.22898256e-02,
       -7.85127506e-02,  5.11369063e-03, -9.41218883e-02, -5.61743490e-02,
       -3.82395908e-02, -1.19635612e-02, -5.22212684e-02, -6.81188237e-03,
        7.69575359e-04,  2.52225883e-02,  2.96068052e-03,  5.83367832e-02,
        6.19569607e-02, -1.02814749e-01, -1.96427908e-02,  9.34371054e-02,
        3.29550961e-03, -2.37782598e-02,  4.09440584e-02, -1.95903797e-02,
       -4.45915535e-02, -1.10558616e-02,  5.59740588e-02,  2.73544993e-02,
       -2.20156088e-02, -2.84697525e-02,  3.47792879e-02, -1.09660521e-01,
        6.82623610e-02,  4.56458554e-02, -6.87996745e-02, -2.81094760e-02,
        1.20403543e-02, -1.53103527e-02, -5.64538166e-02,  5.95052913e-02,
       -5.37956022e-02,  

In [30]:
# Lấy toàn bộ vector, tài liệu và metadata từ collection
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

# Đưa embedding vào mảng numpy
vectors = np.array(result['embeddings'])

# Lưu lại văn bản
documents = result['documents']

# Trích loại tài liệu từ metadata (giả sử có 'doc_type')
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]

# Gán màu sắc tùy theo loại tài liệu
# colors = [['blue', 'green', 'red', 'orange'][['company', 'employees', 'visas', 'schools'].index(t)] for t in doc_types]


In [31]:
color_map = {
    "Quy_che_dao_tao_HaUI": "blue",
    "Chapter I": "green",
    "Chapter II": "red",
    "Chapter III": "orange",
    "Chapter IV": "purple",
    "Chapter V": "brown"
}

colors = [color_map.get(t, "gray") for t in doc_types]  # mặc định gray nếu chưa có

In [32]:
# Con người chúng ta dễ hình dung mọi thứ trong không gian 2D hơn!
# Giảm số chiều của vector xuống 2D bằng t-SNE
# (T-distributed Stochastic Neighbor Embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Tạo biểu đồ scatter 2D
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Loại: {t}<br>Văn bản: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Biểu đồ 2D Chroma Vector Store',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show(renderer="browser")

In [33]:
from langchain.memory import ConversationBufferWindowMemory  
from langchain.chains import ConversationalRetrievalChain

In [34]:
# Tạo mô hình Chat với OpenAI
llm = ChatOpenAI(
    temperature=0.7, 
    model_name=MODEL,
)

# Thiết lập bộ nhớ hội thoại
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# Tạo retriever từ vector store (Chroma)
retriever = vectorstore.as_retriever()

# Kết nối tất cả thành một chuỗi hội thoại có khả năng truy xuất (RAG pipeline)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


C:\Users\admin\AppData\Local\Temp\ipykernel_13052\2637877394.py:8: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [35]:
# Test query with performance monitoring
def test_query_performance():
    """Test query with timing"""
    query = "Bạn có thể mô tả ngắn gọn về cách tính điểm trung bình chung không?"
    start_time = time.time()
    result = conversation_chain.invoke({"question": query})
    end_time = time.time()
    
    print(f"Query processed in {end_time - start_time:.2f} seconds")
    print("Answer:", result["answer"])
    if "source_documents" in result:
        print(f"Used {len(result['source_documents'])} source documents")

In [36]:
test_query_performance()

Query processed in 2.73 seconds
Answer: Tôi không biết.


In [37]:
# set up a new conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [38]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [39]:
# And in Gradio:

# view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

In [40]:
# Hãy cùng tìm hiểu xem điều gì được gửi phía sau hậu trường

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)




In [41]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)


In [42]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]


In [43]:

# view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

# 3 Improve RAG Ensemble Hybrid Retrieval.

In [44]:
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever
from typing import List, Dict

# 1. Fix keyword retriever to search content, not just title
class KeywordRetriever(BaseRetriever):
    context_dict: Dict[str, str]

    def get_relevant_documents(self, query: str) -> List[Document]:
        relevant_docs = []
        for title, content in self.context_dict.items():
            if any(kw in content.lower() for kw in query.lower().split()):
                relevant_docs.append(Document(page_content=content, metadata={"source": title}))
        return relevant_docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return self.get_relevant_documents(query)

# 2. Use keyword + properly configured vector retriever
keyword_retriever = KeywordRetriever(context_dict=ctx)
vector_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 30}
)

# 3. Ensemble
hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

# 4. Conversation chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=hybrid_retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)


C:\Users\admin\AppData\Local\Temp\ipykernel_13052\303262969.py:6: DeprecationWarning:

Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`

C:\Users\admin\AppData\Local\Temp\ipykernel_13052\303262969.py:6: DeprecationWarning:

Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`



In [45]:
query = "Sinh viên được trường xét và công nhận tốt nghiệp khi có đủ các điều kiện nào?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
. Nhà trường sẽ không giải quyết các trường hợp nộp đơn muộn.

5. Việc xếp loại theo mức điểm X được áp dụng đối với những học phần mà phòng đào tạo của trường chưa nhận được báo cáo kết quả học tập của sinh viên từ khoa chuyển lên.

6. Ký hiệu R được áp dụng cho các trường hợp sau:

a) Điểm học phần được đánh giá ở các mức điểm A, B, C, D trong đợt đánh giá đầu học kỳ (nếu có) đối với một số học phần được phép thi sớm để giúp sinh viên học vượt.

b) Những học phần được công nhận, bảo lưu kết quả khi sinh viên chuyển từ trường khác đến hoặc chuyển đổi giữa các chương trình.

7. Các quy định đối với học phần Giáo dục thể chất, Giáo dục Quốc phòng



In [46]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
